# 线性回归的简洁实现
:label:`sec_linear_concise`

在过去的几年里，出于对深度学习强烈的兴趣，许多公司、学者和业余爱好者开发了各种成熟的开源框架。通过这些框架可以自动化实现基于梯度的学习算法中重复性的工作。
在 :numref:`sec_linear_scratch` 中，我们只依赖了：（1）通过张量来进行数据存储和线性代数；（2）通过自动微分来计算梯度。实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用，现代深度学习库也为我们实现了这些组件。

在本节中，我们将介绍如何(**通过使用深度学习框架来简洁地实现**) :numref:`sec_linear_scratch` 中的(**线性回归模型**)。

## 生成数据集

与 :numref:`sec_linear_scratch` 中类似，我们首先[**生成数据集**]。


In [4]:
pip install -U d2l

In [5]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [6]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)#生成数据集，y=Xw+b

## 读取数据集

我们可以[**调用框架中现有的API来读取数据**]。我们将 `features` 和 `labels` 作为API的参数传递，并在实例化数据迭代器对象时指定 `batch_size`。此外，布尔值 `is_train` 表示是否希望数据迭代器对象在每个迭代周期内打乱数据。


In [9]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器。"""
    #is_train表示表示希望数据迭代器对象在每个迭代周期内打乱数据
    #data_arrays：表示可以传入多个矩阵，即是将features和labels作为参数
    #TensorDateset:把输入的两类数据进行一一对应；
    #DataLoader:重新排序
    dataset = data.TensorDataset(*data_arrays)
    #*可以对list解开入参，因为features和labels作为API参数传递
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
    #每次随机挑选batch_size个样本，shuffle意思是要不要打乱顺序

In [11]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用 `data_iter` 的方式与我们在 :numref:`sec_linear_scratch` 中使用 `data_iter` 函数的方式相同。为了验证是否正常工作，让我们读取并打印第一个小批量样本。
与 :numref:`sec_linear_scratch` 不同，这里我们使用 `iter` 构造Python迭代器，并使用 `next` 从迭代器中获取第一项。


In [ ]:
# 不能直接从data_iter中获得数据
next(iter(data_iter))#将data_iter用iter()函数转为迭代器，再使用next()函数从迭代器中获取数据

[tensor([[-0.7090, -0.5001],
         [-0.7020, -1.2230],
         [-1.3061, -1.7845],
         [ 0.3258,  0.9002],
         [ 0.3525, -0.1899],
         [ 2.2436,  0.1764],
         [ 0.6269,  0.9438],
         [ 0.9973,  0.2910],
         [ 0.3698,  0.0483],
         [-2.1677,  0.1078]]),
 tensor([[ 4.4798],
         [ 6.9499],
         [ 7.6255],
         [ 1.8088],
         [ 5.5744],
         [ 8.0814],
         [ 2.2349],
         [ 5.2224],
         [ 4.7888],
         [-0.5108]])]

## 定义模型

当我们在 :numref:`sec_linear_scratch` 中实现线性回归时，我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。但是，如果模型变得更加复杂，而且当你几乎每天都需要实现模型时，你会想简化这个过程。这种情况类似于从头开始编写自己的博客。做一两次是有益的、有启发性的，但如果每次你每需要一个博客就花一个月的时间重新发明轮子，那你将是一个糟糕的网页开发者。

对于标准操作，我们可以[**使用框架的预定义好的层**]。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。我们首先定义一个模型变量`net`，它是一个 `Sequential` 类的实例。 `Sequential` 类为串联在一起的多个层定义了一个容器。当给定输入数据， `Sequential` 实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，依此类推。在下面的例子中，我们的模型只包含一个层，因此实际上不需要`Sequential`。但是由于以后几乎所有的模型都是多层的，在这里使用`Sequential`会让你熟悉标准的流水线。

回顾 :numref:`fig_single_neuron` 中的单层网络架构，这一单层被称为 *全连接层*（fully-connected layer），因为它的每一个输入都通过矩阵-向量乘法连接到它的每个输出。


在 PyTorch 中，全连接层在 `Linear` 类中定义。值得注意的是，我们将两个参数传递到 `nn.Linear` 中。第一个指定输入特征形状，即 2，第二个指定输出特征形状，输出特征形状为单个标量，因此为 1。


In [13]:
# `nn` 是神经网络的缩写
from torch import nn
#Sequential 类为串联在一起的多个层定义了一个容器
#当给定输入数据， Sequential 实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，依此类推
net = nn.Sequential(nn.Linear(2, 1))#Linear中，第一个指定输入特征数，第二个指定输出数。
#输入维度是2，输出纬度是1；
#nn.Linear(2,1)可以理解为线性回归就是简单的单层神经网络，将其放在一个Sequential中

## (**初始化模型参数**)

在使用`net`之前，我们需要初始化模型参数。如在线性回归模型中的权重和偏置。
深度学习框架通常有预定义的方法来初始化参数。
在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零。


正如我们在构造 `nn.Linear` 时指定输入和输出尺寸一样。现在我们直接访问参数以设定初始值。我们通过 `net[0]` 选择网络中的第一个图层，然后使用 `weight.data` 和 `bias.data` 方法访问参数。然后使用替换方法 `normal_` 和 `fill_` 来重写参数值。


In [19]:
#net[0]表示使用网络中的第一个图层
#指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零
print(net[0].weight.data)
net[0].weight.data.normal_(0, 0.01)#使用正态分布替代data的值，均值为0，标准差为0.01
print(net[0].weight.data)
print(net[0].bias.data)
net[0].bias.data.fill_(0)
print(net[0].bias.data)

tensor([[ 0.0008, -0.0002]])
tensor([[-0.0015, -0.0049]])
tensor([0.])
tensor([0.])


## 定义损失函数


[**计算均方误差使用的是`MSELoss`类，也称为平方 $L_2$ 范数**]。默认情况下，它返回所有样本损失的平均值。


In [23]:
loss = nn.MSELoss()#计算均方误差使用的是MSELoss类，也称为平方𝐿2范数 mean-square error

## 定义优化算法


小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch 在 `optim` 模块中实现了该算法的许多变种。当我们(**实例化 `SGD` 实例**)时，我们要指定优化的参数（可通过 `net.parameters()` 从我们的模型中获得）以及优化算法所需的超参数字典。小批量随机梯度下降只需要设置 `lr`值，这里设置为 0.03。


In [20]:
#PyTorch 在 optim 模块中实现了该算法的许多变种
#可通过 net.parameters() 从我们的模型中获得指定优化的参数
#SGD随机梯度下降
trainer = torch.optim.SGD(net.parameters(), lr=0.03)#net.parameters()包括w和b

<generator object Module.parameters at 0x7f680d8682d0>


## 训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。
我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。
当我们需要更复杂的模型时，高级API的优势将大大增加。
当我们有了所有的基本组件，[**训练过程代码与我们从零开始实现时所做的非常相似**]。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（`train_data`），不停地从中获取一个小批量的输入和相应的标签。对于每一个小批量，我们会进行以下步骤:

* 通过调用 `net(X)` 生成预测并计算损失 `l`（正向传播）。
* 通过进行反向传播来计算梯度。
* 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。


In [26]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)#net(X)即为前向输出，y_hat
        trainer.zero_grad()#梯度清零
        l.backward()#等价于l.sum().backward()求和之后算梯度
        trainer.step()#调用优化算法进行模型更新
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000103
epoch 2, loss 0.000103
epoch 3, loss 0.000103


下面我们[**比较生成数据集的真实参数和通过有限数据训练获得的模型参数**]。
要访问参数，我们首先从 `net` 访问所需的层，然后读取该层的权重和偏置。
正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。


In [1]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

NameError: ignored

## 小结


* 我们可以使用 PyTorch 的高级 API更简洁地实现模型。
* 在 PyTorch 中，`data` 模块提供了数据处理工具，`nn` 模块定义了大量的神经网络层和常见损失函数。
* 我们可以通过`_` 结尾的方法将参数替换，从而初始化参数。


## 练习


1. 如果我们用 `nn.MSELoss()` 替换 `nn.MSELoss(reduction='sum')`，为了使代码的行为相同，需要怎么更改学习速率？为什么？
1. 查看 PyTorch 文档，了解提供了哪些损失函数和初始化方法。用Huber损失来代替。
1. 你如何访问 `net[0].weight` 的梯度？

[Discussions](https://discuss.d2l.ai/t/1781)
